# Vectorizing Raw Data: N-Grams

### N-Grams

Creates a document-term matrix where counts still occupy the cell but instead of the columns representing single terms, they represent all combinations of adjacent words of length n in your text.  
(Google's autocomplete uses n-grams like approach)

"NLP is an interesting topic"

| n   | Name      | Tokens                                                     |
| --- | --------- | ---------------------------------------------------------- |
| 2   | bigram    | ["nlp is", "is an", "an interesting", "interesting topic"] |
| 3   | trigram   | ["nlp is an", "is an interesting", "an interesting topic"] |
| 4   | four-gram | ["nlp is an interesting", "is an interesting topic"]       |


### Read in text

In [1]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

### Create function to remove punctuation, tokenize, remove stopwords, and stem

In [2]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    return text

data['cleaned_text'] = data['body_text'].apply(lambda x: clean_text(x))
data.head()

,label,body_text,cleaned_text
0,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri questionstd...
1,ham,"Nah I don't think he goes to usf, he lives around here though",nah dont think goe usf live around though
2,ham,Even my brother is not like to speak with me. They treat me like aids patent.,even brother like speak treat like aid patent
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,date sunday
4,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your call...,per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend...


### Apply CountVectorizer (w/ N-Grams)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

ngram_vect = CountVectorizer(ngram_range=(2, 2))
# only bigrams
# (1, 1) - only unigrams (default), (1, 2) - unigrams and bigrams, (1, 3) - unigrams, bigrams, trigrams

X_counts = ngram_vect.fit_transform(data['cleaned_text'])
print(X_counts.shape)
print(ngram_vect.get_feature_names()[-100:])

(5567, 31260)
['youso much', 'youthat give', 'youto find', 'youuuuu go', 'youv alreadi', 'youv alway', 'youv charg', 'youv come', 'youv got', 'youv live', 'youv said', 'youv settl', 'youv tkt', 'youv wonder', 'youv your', 'youwanna chat', 'youwhen wil', 'yovil phone', 'yowif hint', 'yoyyooo know', 'yr area', 'yr buddi', 'yr chik', 'yr choic', 'yr difficult', 'yr draw', 'yr eve', 'yr give', 'yr need', 'yr network', 'yr phone', 'yr prize', 'yr project', 'yr say', 'yr suppli', 'yr term', 'ystrdayic medicin', 'yummi babe', 'yummi lip', 'yun ahnow', 'yun ahth', 'yun buy', 'yun dun', 'yun fetch', 'yunni im', 'yuo exmpel', 'yuo ra', 'yuou work', 'yup alreadi', 'yup anyth', 'yup bath', 'yup come', 'yup comin', 'yup cut', 'yup fren', 'yup give', 'yup go', 'yup havent', 'yup hey', 'yup hope', 'yup im', 'yup ive', 'yup izzit', 'yup leav', 'yup lunch', 'yup msg', 'yup need', 'yup next', 'yup noe', 'yup ok', 'yup paragon', 'yup remb', 'yup send', 'yup shd', 'yup song', 'yup studi', 'yup thk', 'yup 

### Apply CountVectorizer (w/ N-Grams) to smaller sample

In [5]:
data_sample = data[0:20]

ngram_vect_sample = CountVectorizer(ngram_range=(2, 2))

X_counts_sample = ngram_vect_sample.fit_transform(data_sample['cleaned_text'])
print(X_counts_sample.shape)
print(ngram_vect_sample.get_feature_names()[-100:])

(20, 198)
['link next', 'live around', 'lor ard', 'lor finish', 'lunch alreadi', 'lunch go', 'macedonia dont', 'make wet', 'may 2005', 'meet sooner', 'mell mell', 'mell oru', 'membership 100000', 'messag click', 'minnaminungint nurungu', 'miss goalsteam', 'mobil 11', 'mobil camera', 'mobil updat', 'month entitl', 'month ha', 'nah dont', 'name ye', 'nation team', 'naughti make', 'network custom', 'news txt', 'next txt', 'nurungu vettam', 'oh kim', 'oru minnaminungint', 'pay first', 'per request', 'pobox 4403ldnw1a7rw18', 'poboxox36504w45wq 16', 'pound txt', 'press copi', 'prize jackpot', 'prize reward', 'questionstd txt', 'ratetc appli', 'receiv entri', 'receivea 900', 'rememb spell', 'repli hl', 'request mell', 'reward claim', 'scotland 4txtú120', 'select receivea', 'send 87575', 'serious spell', 'set callertun', 'six chanc', 'smth lor', 'soon dont', 'speak treat', 'spell name', 'stock comin', 'str lor', 'stuff anymor', 'talk stuff', 'tc wwwdbuknet', 'team 87077', 'text fa', 'think goe

### Vectorizers output sparse matrices

_**Sparse Matrix**: A matrix in which most entries are 0. In the interest of efficient storage, a sparse matrix will be stored by only storing the locations of the non-zero elements._

In [10]:
X_counts_df = pd.DataFrame(X_counts_sample.toarray())

X_counts_df.columns = ngram_vect_sample.get_feature_names()
X_counts_df

,09061701461 claim,100 20000,100000 prize,11 month,12 hour,150pday 6day,16 tsandc,20000 pound,2005 text,21st may,...,way meet,week free,win cash,win fa,winner valu,wkli comp,word claim,wwwdbuknet lccltd,xxxmobilemovieclub use,ye naughti
0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0
